In [2]:
import json
import re

In [3]:
#renvoie un set de stopwords
with open("stopwords-fr.json",encoding="utf-8") as f:
    set_stop = set(json.load(f))

In [4]:
def filter_KW(liste):
  out = []
  #on enlève tous les stopwords (liste resultante)
  #filtrage des tokens qui sont courts ou des entiers
  liste = set(liste).difference(set_stop)
  for x in liste:
    numbers = re.findall("[0-9]",x)
    if x.lower() in set_stop:
      continue
    elif len(x)<2:
      continue
    elif len(re.findall("[A-Za-z]", x))==0:
      continue
    elif len(numbers)>0:
       if len(numbers[0])==len(x):
         continue
    out.append(x)
  return out

In [5]:
def tokenize(chaine):
  chaine = re.sub("\.|'|\?|»|«|\"", " ", chaine)
  chaine = re.sub(" {2,}"," ", chaine)
  mots = chaine.split()
  return mots

In [6]:
def is_subsequence(needle, haystack):
  return any( haystack[i:i+len(needle)] == needle
            for i in range(len(haystack) - len(needle) + 1))

In [7]:
#inter : ce qui est à la fois dans titre et body
#last : indice
def get_ngrams(inter, word_titre, word_body):
 
  selected = [x for x in inter]
  haystack=[x.lower() for x in word_body]
  #le mot est dans le titre et dans le chapeau
  for i, word in enumerate(word_titre):
    if word in inter or word.lower() in inter:
      n_gram = [word]
      last = i
      for j, w in enumerate(word_titre[i+1:]):
        if w in inter:
          last = j
        
        n_gram.append(w)
      if len(n_gram)>1 and len(n_gram)<5 and last!=i:
        needle = [x.lower() for x in n_gram]
        if is_subsequence(needle, haystack):
          selected.append("_".join(n_gram[:last+2]))
        
## TODO: régler le problème des n-grammes
##  if "gilets" in haystack and "jaunes in haystack":
##    if "gilets_jaunes" not in selected:
##      print(selected)
##      x = haystack.index("gilets")
##      print(haystack[x:x+10])
##      print(word_titre[word_titre.index("gilets"):])
  return selected


In [8]:
def get_KW(titre, text):
  body = text
  chapeau = ""

  if len(titre)<100:
    chapeau = text[:200]
    body = text[200:]

  word_titre = set(tokenize(titre) + tokenize(chapeau))
  word_body = set(tokenize(body))
  inter = word_titre.intersection(word_body)
  inter = filter_KW(inter)
  word_titre_2 = tokenize(titre) + tokenize(chapeau)
  inter = get_ngrams(inter, tokenize(titre), tokenize(body))
  return inter

In [9]:
with open("corpus_final/corpus.json") as f:
  L = json.load(f)

In [10]:
dic = {"index":{}, "index_I":{}}

for cpt, info_text in enumerate(L):
   
  titre = info_text["titre"]
  text = info_text["texte"]
  date = info_text["date"]
  elems_date = re.findall("[0-9]{1,4}", date)
  
  if len(elems_date[0])==2:
    L[cpt]["date"] = "%s-%s-%s"%(elems_date[1], elems_date[2], elems_date[0])
  liste_KW = get_KW(titre, text)

  
  for KW in liste_KW:
    if KW.lower() in dic["index"]:
      KW = KW.lower()
    dic["index"].setdefault(KW, [])
    #pour le mot-clé, on lui associe sa ou ses position(s) (article dont il est issu)
    dic["index"][KW].append(cpt)
    dic["index_I"].setdefault(cpt, [])
    dic["index_I"][cpt].append(KW)

print("On a %i Documents"%(cpt+1))
#on calcule le nombre d'articles où apparaît un mot-clé et on le range dans une liste de listes
l_test = [[len(dic["index"][kw]), kw] for kw in dic["index"]]

#tri des occurrences de mot-clé
S = sorted([x for x in l_test if x[0]>1], reverse=True)
#tri des occurrences de mot-clés qui sont des n-grammes, n>1
S2 = sorted([x for x in l_test if "_" in x[1]], reverse=True)

top10 = [x[1] for x in S[:10]]
top20 = [x for x in S[:20]]
print("Top20 :",top20)

topGrams = [x for x in S2[:10]]
print("Top grams :",topGrams)

entetes = ["date", "journal", "titre"]
out_csv = [[x for x in entetes]]

for i in range(len(top10)):
  out_csv[0].append(top10[i])
out_csv[0].append("autres mots-clés")
out_csv[0] = ";".join(out_csv[0])


for ID_txt, liste_KW in dic["index_I"].items(): #cpt,mots-clés
  #recuperer texte
  info_txt = L[ID_txt]
  
  ligne = [info_txt[cle] for cle in entetes]+[""]*10+[[]]
  
  for kw in liste_KW:
    if kw in top10:
      ligne[top10.index(kw)+len(entetes)]="X"
    else:
      ligne[-1].append(kw)
  if len(ligne[2])>100:
    ligne[2] = ligne[2][:100]+"..."
  ligne[-1]=" , ".join(ligne[-1])
  ligne = [re.sub(";", " ", x) for x in ligne]
  out_csv.append(";".join(ligne))

with open("out_complet.csv", "w",encoding="utf-8") as w:
  w.write("\n".join(out_csv))


On a 3330 Documents
Top20 : [[192, 'complot'], [159, 'France'], [138, 'monde'], [122, 'Trump'], [119, 'théories'], [119, 'politique'], [117, 'président'], [101, 'extrême'], [76, 'Donald'], [67, 'français'], [64, 'gilets'], [63, 'mouvement'], [63, 'crise'], [63, 'affaire'], [62, 'jeunes'], [60, 'pays'], [60, 'jaunes'], [60, 'enquête'], [59, 'réseaux'], [58, 'ministre']]
Top grams : [[13, 'réseaux_sociaux'], [12, 'gilets_jaunes'], [6, 'théories_du_complot'], [5, 'autorités_constituées'], [5, 'Donald_Trump'], [4, 'théorie_du_complot'], [4, 'liberté_de_la_presse'], [4, 'fake_news'], [4, 'attaque_de_Strasbourg'], [3, 'éducation_nationale']]


In [11]:
#-----------------RAKE PYTHON-------------------------------#

In [12]:
#!pip install multi-rake

In [13]:
from multi_rake import Rake
import re

#renvoie les mot-clés qui apparaissent dans le plus d'articles
def top15_corpus_rake(L,n_grams) :
    dico_mots_cles = {}
    r = Rake(language_code='fr',max_words=n_grams)
    for element in L :
        mot_cles = r.apply(element['texte'])
        i = 0
        #dictionnaire qui calcule le nombre d'occurrences des mots-clés pour tout le corpus
        for mot_cle in mot_cles :
            if mot_cle[0] not in dico_mots_cles :
                dico_mots_cles[mot_cle[0]] = 0
            dico_mots_cles[mot_cle[0]] += 1
            i+=1
            if i == 15 :
                break
        
    dico_trie_mots_cles = dict(sorted(dico_mots_cles.items(), key=lambda item:item[1],reverse=True))
    return dico_trie_mots_cles

dico_kw_rake = {}
for i in range(1,4) :
    
    dico_trie_mots_cles = top15_corpus_rake(L,i)
    
    j = 0
    for k,v in dico_trie_mots_cles.items() :
        #on obtient les mots clés et leur nombre d'occurrences dans les articles
        k2 = re.sub('\s','_',k)
        if k2 not in dico_kw_rake :
            dico_kw_rake[k2] = v
        if k2 in dico_kw_rake and v > dico_kw_rake[k2] :
            dico_kw_rake[k2] = v
        #liste.append((k2,v))
        j+=1
        
        if j == 15 :
            #print(liste)
            break
        
print("Les mots-clés du corpus (Rake) sont : ")
#dico renvoie les 15 mots-clés pour chaque n-gram et pour chaque mot-clé, prend l'occurrence d'articles
#la plus grande (avec rake, les mots-clés par article changent selon le max_words choisi)
print(dico_kw_rake)
#print(len(dico_kw_rake))


Les mots-clés du corpus (Rake) sont : 
{"c'est": 406, 'france': 248, 'complot': 199, 'monde': 199, 'théories': 168, 'paris': 152, 'pays': 117, 'temps': 109, 'complotisme': 108, "n'est": 107, 'président': 102, 'attentats': 101, 'etats-unis': 100, 'médias': 99, 'français': 92, 'réseaux_sociaux': 239, 'gilets_jaunes': 132, 'fake_news': 96, 'new_york': 92, 'front_national': 67, 'donald_trump': 66, 'charlie_hebdo': 65, 'quelques_jours': 57, 'emmanuel_macron': 55, "l'union_européenne": 52, 'premier_ministre': 47, "l'élection_présidentielle": 42, 'fausses_informations': 38, 'fondation_jean-jaurès': 37, 'dix_ans': 37, 'new_york_times': 73, 'seconde_guerre_mondiale': 62, 'world_trade_center': 42, 'da_vinci_code': 36, 'site_conspiracy_watch': 34, 'première_guerre_mondiale': 23, 'recep_tayyip_erdogan': 21, 'ku_klux_klan': 19}


In [14]:
#calcul des mots-clés par article
def mots_cles_article(texte) :
    from operator import itemgetter
    L_mots_cles = []
    #texte = L[i]['texte']
    for n_grams in range(1,4) :
        mots_cles = []
        
        r = Rake(language_code='fr',max_words=n_grams)
        mots_cles = r.apply(texte)
        tri_mots_cles = sorted(mots_cles, key=itemgetter(1),reverse = True)
        #on parcourt la liste de mots clés pour avoir une liste de tuples à la fin (et pas une liste de listes)
        for i in range(5) :
            L_mots_cles.append(re.sub('\s','_',tri_mots_cles[i][0]))
   
    L_triee = sorted(L_mots_cles, key=itemgetter(1),reverse = True)
    return L_triee[:15]
for i in range(10) :
    print(mots_cles_article(L[i]['texte']))
    print("-"*100)

["susceptibles_d'y_adhérer", "culturel_d'une_extrême", 'étoiles_juives_associées', 'profiter', 'rothschild', "analyse_l'historienne_spécialiste", 'vidéos_accusant', 'rhétorique_conspirationniste', 'destiné', 'responsables_désignés', 'semaines_autour', 'relents_souvent_antisémites', 'pandémie', "l'épidémie", "d'autres_affirmant"]
----------------------------------------------------------------------------------------------------
["qu'il", 'rudy_reichstadt', 'succès_massif', 'créé', "grave_erreur_d'analyse", 'cons_pirationnisme', 'fondation_jean_jaurès', 'directeur', 'site_conspiracy_watch', 'virage_éditorial_favorable', 'membre', 'radicalités_politiques', 'faisant_soigneusement_oublier', 'a-t-il_surpris', "l'observatoire"]
----------------------------------------------------------------------------------------------------
['gravité', 'premier_ministre', 'président_emmanuel_macron', 'confinement_partiel', 'annoncé', 'limité', 'pic_actuel', 'vingt_ans_énormément', 'temps', 'jean_castex', 

In [15]:
#------------------comparaison mots clés corpus entre Daniel et rake---------------------#

strong_kw = set()

#on recupère les mots-clés détectés par daniel (unigrammes et n-grammes) et on les stocke dans un ensemble
daniel_kw = top20+topGrams
daniel_kw_set = set()

#mettre tout en minuscules car rake met tout en minuscule
for d in daniel_kw :
    daniel_kw_set.add(d[1].lower())

#on récupère les mots-clés détectés par rake
rake_kw_set = set()
for k,v in dico_kw_rake.items() :
    rake_kw_set.add(k)

strong_kw = daniel_kw_set.intersection(rake_kw_set)
print("Les mots-clés du corpus qui apparaissent à la fois avec daniel et rake sont : ",strong_kw)

Les mots-clés du corpus qui apparaissent à la fois avec daniel et rake sont :  {'pays', 'monde', 'fake_news', 'français', 'réseaux_sociaux', 'président', 'donald_trump', 'théories', 'complot', 'gilets_jaunes', 'france'}


In [16]:
#Fonction qui renvoie les autres mots clés communs aux deux systèmes et qui ne sont pas des mots-clés du corpus
def strong_other_kw(L,i,strong_kw) :
    #on récupère les mots-clés d'un article, avec les deux systèmes
    daniel_kw = get_KW(L[i]['titre'],L[i]['texte'])
    rake_kw_set = set(mots_cles_article(L[i]['texte']))
    
    #on recupère les mots-clés détectés par daniel (unigrammes et n-grammes) et on les stocke dans un ensemble
    daniel_kw_set = set()
    for d in daniel_kw :
        daniel_kw_set.add(d.lower())
    
    #commun est l'intersection des mots-clés de daniel et rake

    commun = set()
    commun = daniel_kw_set.intersection(rake_kw_set)
    #print(commun)
    
    #strong_other_kw est l'ensemble des mots-clés détectés par daniel et rake et qui ne sont pas des mots-clés
    #du corpus
    strong_other_kw = set()
    strong_other_kw = commun.difference(strong_kw)
    return strong_other_kw


cpt = 0
for i in range(len(L)) :
    if len(strong_other_kw(L,0,strong_kw)) > 0 :
        print(strong_other_kw(L,i,strong_kw))
        cpt+=1
print("Le nombre de mots-clés autres détectés par Daniel et Rake est : ",cpt)

Le nombre de mots-clés autres détectés par Daniel et Rake est :  0
